# Final Project
I took inspiration from my first project and am attempting to expound upon it. I used all the columns to predict one feature in the previous iteration, this time I am taking 1 column (a string representation of the molecule) and attempting to predict the other columns. I am using a multilayer perceptron regressor as the output should be continous values.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('Molecules.csv')

df.head()

,row ID,Molecule,Molecule name,Mannhold LogP,Atomic Polarizabilities,Aromatic Atoms Count,Aromatic Bonds Count,Element Count,Bond Polarizabilities,Bond Count,...,XLogP,Zagreb Index,Molecular Formula,Formal Charge,Formal Charge (pos),Formal Charge (neg),Heavy Atoms Count,Molar Mass,SP3 Character,Rotatable Bonds Count (non terminal)
0,Row0,N,H3N,1.35,3.100379,0,0,4,1.299621,0,...,0.000,0,H3N,0,0,0,1,17.0305254685134,0.000000,0
1,Row1,O=C(O)c1c(OC(=O)C)cccc1,C9H8O4,2.01,24.382344,6,6,21,12.577656,13,...,1.422,60,C9H8O4,0,0,0,13,180.157768811976,0.047619,3
2,Row2,C#C,C2H2,1.68,4.853586,0,0,4,2.186414,1,...,0.418,2,C2H2,0,0,0,2,26.0373533012276,0.000000,0
3,Row3,P(=O)(OP(=O)(OC[C@H]1O[C@@H](n2c3ncnc(c3nc2)N)...,C10H16N5O13P3,0.25,55.084688,9,10,47,55.749312,33,...,-5.533,170,C10H16N5O13P3,0,0,0,31,507.181476012675,0.106383,8
4,Row4,S1C([C@H](C(=O)O)N2[C@H]1[C@H](NC(=O)[C@H](N)c...,C16H19N3O4S,2.34,50.237067,6,6,43,27.904933,26,...,-1.732,132,C16H19N3O4S,0,0,0,24,349.406462995722,0.162791,4


In [3]:
df = df[df.apply(lambda row: not any(row == '#NUM!'), axis=1)]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13600 entries, 0 to 15165
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   row ID                                  13600 non-null  object 
 1   Molecule                                13600 non-null  object 
 2   Molecule name                           13600 non-null  object 
 3   Mannhold LogP                           13600 non-null  float64
 4   Atomic Polarizabilities                 13600 non-null  object 
 5   Aromatic Atoms Count                    13600 non-null  int64  
 6   Aromatic Bonds Count                    13600 non-null  int64  
 7   Element Count                           13600 non-null  int64  
 8   Bond Polarizabilities                   13600 non-null  object 
 9   Bond Count                              13600 non-null  int64  
 10  Eccentric Connectivity Index            13600 non-null  int64  

In [4]:
! pip install transformers

^C


  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for fsspec>=2023.5.0 from https://files.pythonhosted.org/packages/70/25/fab23259a52ece5670dcb8452e1af34b89e6135ecc17cd4b54b4b479eac6/fsspec-2023.12.2-py3-none-any.whl.metadata
Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
   ---------------------------------------- 0.0/169.0 kB ? eta -:--:--
   --------- ------------------------------ 41.0/169.0 kB 2.0 MB/s eta 0:00:01
   ----------------------------- ---------- 122.9/169.0 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 169.0/169.0 kB 1.7 MB/s eta 0:00:00


In [5]:
import requests
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
smiles = df['Molecule']
tokenizer.fit_on_texts(smiles)
sequences = tokenizer.texts_to_matrix(smiles, mode='binary')

print(sequences)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 0. 0. 0.]]


In [22]:
pd.DataFrame(sequences)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13595,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13596,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13597,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Preprocessing
- created copy of data set
- removed rows with non numerical data
- removed duplicate rows
- normalized dataset

In [26]:
working_data = df[:]

In [27]:
#Removing columns with nonnumerical data
del working_data['row ID']
del working_data['Molecule name']
smiles = df['Molecule']
del working_data['Molecule']
del working_data['Molecular Formula']
#perfectly correlated with bond count so removing
del working_data['Zagreb Index']
#Molar mass and molecular weight are the same thing so one was removed
del working_data['Molar Mass']
#The rest were removed becuase they aren't as imprtant or are not difficult to calculate.
#The hope was that with less outputs, the model would be more accurate
del working_data['Eccentric Connectivity Index']
del working_data['Formal Charge (pos)']
del working_data['Formal Charge (neg)']
del working_data['Heavy Atoms Count']
del working_data['Hydrogen Bond Acceptors']
del working_data['Hydrogen Bond Donors']
del working_data['Rotatable Bonds Count (non terminal)']
del working_data['Rotatable Bonds Count']
del working_data['Lipinski\'s Rule of Five']
del working_data['Atomic Polarizabilities']
del working_data['Aromatic Atoms Count']
del working_data['Aromatic Bonds Count']
del working_data['Largest Pi Chain']
del working_data['Largest Chain']


In [29]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(sequences, working_data, test_size=0.25, random_state=42)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
}

mlp_regressor = MLPRegressor(hidden_layer_sizes=(100,50), max_iter=1000, random_state=42)
mlp_regressor.fit(X_train, y_train)

# Make predictions on the test set
predictions = mlp_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 32668360.71791094


In [33]:
scores = cross_val_score(mlp_regressor, sequences, working_data, cv=5, n_jobs=-1)

print("Cross-Validated Scores:", scores)

# Print the mean score
print("Mean Score:", scores.mean())

Cross-Validated Scores: [  -1.10682545   -1.56445269  -84.91037516  -38.19254291 -177.26757023]
Mean Score: -60.608353286598


In [31]:
y_test

,Mannhold LogP,Aromatic Bonds Count,Element Count,Bond Polarizabilities,Bond Count,Fragment Complexity,VABC Volume Descriptor,Petitjean Number,Topological Polar Surface Area,Vertex adjacency information magnitude,Molecular Weight,XLogP,Formal Charge,SP3 Character
12272,2.56,0,36,23.239312,22,1064.05,261.695691985206,0.428571,72.83,5.459432,276.099773612,0.343,0,0.194444
7096,2.01,6,18,11.094863,9,252.02,118.249503187395,0.400000,35.25,4.169925,123.068413908,1.120,0,0.055556
6571,3.66,0,68,52.061108,25,4209.02,386.348723484603,0.500000,6.48,5.643856,336.350449408001,7.075,0,0.323529
106,2.01,10,36,31.578519,20,1027.07,236.031202632978,0.444444,95.6,5.321928,284.092593654,0.206,0,0.111111
6848,2.34,6,37,25.390933,18,1063.05,249.312848972301,0.444444,73.1,5.169925,245.164045608,2.191,0,0.162162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269,3.88,22,87,59.447487,49,6030.12,587.61213432584,0.500000,154.03,6.614710,627.316917412001,5.718,0,0.137931
5299,1.68,0,31,24.882105,15,660.07,228.879557842567,0.500000,125.49,4.906891,250.06062483,0.009,0,0.193548
4862,0.80,0,16,19.511242,9,135.08,132.9078362711,0.400000,125.5,4.169925,189.960579912,-2.014,0,0.125000
12353,2.78,17,63,45.139175,42,3083.13,437.676730123222,0.500000,101.12,6.392317,534.172619180001,6.785,0,0.158730


In [32]:
pd.DataFrame(predictions, columns=y_test.columns)

,Mannhold LogP,Aromatic Bonds Count,Element Count,Bond Polarizabilities,Bond Count,Fragment Complexity,VABC Volume Descriptor,Petitjean Number,Topological Polar Surface Area,Vertex adjacency information magnitude,Molecular Weight,XLogP,Formal Charge,SP3 Character
0,3.352932,9.701258,50.395273,31.764043,29.422168,2706.834698,345.761190,0.510985,79.334812,5.877457,372.130506,3.750812,0.039348,0.058373
1,2.300240,5.260033,24.232079,15.698223,13.825786,627.058770,174.115515,0.413203,44.609962,4.568961,193.927907,1.837332,0.013729,0.052762
2,2.397671,5.699488,25.745718,16.524972,14.783610,698.721946,181.106055,0.407979,44.087561,4.725453,200.819239,2.054074,-0.008952,0.063159
3,3.184770,7.714591,55.013886,36.769979,30.691578,3847.587046,365.232371,0.467902,95.591428,5.395993,391.975762,2.916245,0.177139,0.116232
4,2.458222,6.847643,31.048354,20.338278,18.299791,1151.932737,224.364395,0.498955,59.762023,4.696750,248.910560,2.341347,0.116406,0.059290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,4.633673,15.080623,83.969197,53.716378,49.336840,5726.222608,553.423856,0.558334,117.159068,7.192598,586.022869,6.041069,0.133578,0.049363
3396,2.379284,4.338235,27.148347,18.358568,14.760556,1107.832111,184.950498,0.435214,45.764258,4.376523,202.945919,1.899856,0.085139,0.106666
3397,1.967295,3.076632,17.787243,12.587602,9.397333,369.939666,126.825502,0.384618,35.146848,3.923437,143.419173,1.158300,0.065820,0.079267
3398,3.597177,15.119705,64.817339,40.889050,40.469534,3638.585396,476.607735,0.743364,126.379045,6.297981,520.408153,4.651015,0.291879,-0.050056


# Conclusions
The model isn't as accurate as I had hoped. I think that the method for encoding the SMILEs format was not the most effective. There was a different package that was meant to handle SMILEs but i was unable to get the package working. I resorted to just one-hot encoding and think that there was not enough information for the model to train properly. The estimates are close but never accurate enough. The model serves to give an idea of values but values should not be taken as accurate. More work would need to be done to better represent the molecules, a way that contains more data than the current method. That being said it is still interesting to see the ability of a neural network to predic 14 different values given one string broken into 27 columns as input. I think it shows how mathematical the world is and chemistry as well. 